In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet")

val  = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet')

test = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet')

In [3]:
train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2022-01-01 00:14:21,2022-01-01 00:15:33,N,1.0,42,42,1.0,0.44,3.5,0.50,0.5,0.00,0.0,None,0.3,4.80,2.0,1.0,0.00
1,1,2022-01-01 00:20:55,2022-01-01 00:29:38,N,1.0,116,41,1.0,2.10,9.5,0.50,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.00
2,1,2022-01-01 00:57:02,2022-01-01 01:13:14,N,1.0,41,140,1.0,3.70,14.5,3.25,0.5,4.60,0.0,None,0.3,23.15,1.0,1.0,2.75
3,2,2022-01-01 00:07:42,2022-01-01 00:15:57,N,1.0,181,181,1.0,1.69,8.0,0.50,0.5,0.00,0.0,None,0.3,9.30,2.0,1.0,0.00
4,2,2022-01-01 00:07:50,2022-01-01 00:28:52,N,1.0,33,170,1.0,6.26,22.0,0.50,0.5,5.21,0.0,None,0.3,31.26,1.0,1.0,2.75


In [4]:
dv = DictVectorizer()

In [5]:
def read_dataframe(df: pd.DataFrame):
    #df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

In [6]:
def data_processing(train_df, val_df, test_df):
    df_train = read_dataframe(train_df)
    df_val = read_dataframe(val_df)
    df_test = read_dataframe(test_df)

    target = 'tip_amount'

    y_train = df_train[target].values
    y_val= df_val[target].values
    y_test = df_test[target].values

    """Init the dictvectorizer""" 
    dv = DictVectorizer()
    X_train, dv = preprocess(df_train, dv, fit_dv=True)
    X_val, _ = preprocess(df_val, dv, fit_dv=False)
    X_test, _ = preprocess(df_test, dv, fit_dv=False)

    return X_train,y_train,X_val,y_val,X_test,y_test,dv

    

In [8]:
X_train,y_train,X_val,y_val,X_test,y_test,dv = data_processing(train, val, test)

/tmp/ipykernel_2377/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)
/tmp/ipykernel_2377/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)
/tmp/ipykernel_2377/1979658928.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [9]:
#save the dictvectorizer
import pickle
with open('2_homework/dv.pkl', 'wb') as file:
    pickle.dump(dv, file)

answer to the Q2

In [10]:
#To check the size of the file
!du -h "/home/ubuntu/notebooks/2_homework/dv.pkl"

#the answer is b-154kb

152K	/home/ubuntu/notebooks/2_homework/dv.pkl


------------------------------------------------

Q3 Train a model with autolog

In [10]:
def train_func(X_train,y_train,X_val,y_val,model):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_pred,y_val)
    return rmse


In [13]:
model  = RandomForestRegressor(max_depth=10, random_state=0)

In [15]:
mlflow.set_tracking_uri(uri="http://localhost:5000")
mlflow.set_experiment("2_homework")



<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>

In [36]:
with mlflow.start_run():
    mlflow.set_tag('Model','RandomForestRegressor')
    mlflow.sklearn.autolog()

    #run the model
    rmse = train_func(X_train,y_train,X_val,y_val,model)

    #run the metric logging
    mlflow.log_metric('RMSE',rmse)

    #log the model artifact
    mlflow.sklearn.log_model(model, artifact_path='models')

2024/02/02 06:46:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


In [16]:
# now we have been tasked to check the max_depth of the model
# we will try it using mlflow client

from mlflow.client import MlflowClient

client = MlflowClient(tracking_uri=mlflow.get_tracking_uri())

In [17]:
client.search_experiments()
#This lists all the experiments

[<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1706710158792, experiment_id='4', last_update_time=1706710158792, lifecycle_stage='active', name='lightgbm prediction experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1706708938289, experiment_id='3', last_update_time=1706708938289, lifecycle_stage='active', name='2_model_registry', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/2', creation_time=1705592485729, experiment_id='2', last_update_time=1705592485729, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/1', creation_time=1705591891060, experiment_id='1', last_update_time=1705591891060, lifecycle_stage='active', name='nyc-taxi-exper

This answers the 3rd question

In [18]:
runs = client.search_runs(
    experiment_ids=5
)


#since we know our experiment id is 5

for run in runs:
    print(f"the max_depth is {run.data.params['max_depth']}")





the max_depth is 10


===================================================================

# Q4

In [ ]:
#%pip install optuna
#first  lets define the objective function to try different values to tune using optuna 
# then define the objective function


#import the necessary libs



In [9]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

  #init the mode

In [10]:



def objective(trial):
    #define the hyperparams
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 50, 1),
        'max_depth': trial.suggest_int('max_depth', 1, 20, 1),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10, 1),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4, 1),
        'random_state': 42,
        'n_jobs': -1
    }
    #model and features and targets have been initialized following previous experiments
    model = RandomForestRegressor(**params)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    return rmse



In [11]:
#Conduct the study and store the params in the study object
sampler = optuna.samplers.TPESampler()
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective,n_trials=10)

[I 2024-02-02 15:26:32,425] A new study created in memory with name: no-name-06895668-16ef-46d5-9418-cc949edca53f
/tmp/ipykernel_2377/4193407549.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 10, 50, 1),
/tmp/ipykernel_2377/4193407549.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 1, 20, 1),
/tmp/ipykernel_2377/4193407549.py:6: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'min_samples_split': trial.suggest_int('min_samples_split', 2, 10, 1),
/tmp/ipykernel_2377/4193407549.py:7: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'min_samples_leaf': trial.suggest_int('min_samples

In [12]:
trials=study.get_trials() #get all the trials conducted


In [45]:
trials[0].values

[2.4500218239093114]

 Now for each trial conduct an exp


In [14]:
#before that we will need to delete the existing deleted experiment permanently


#import sqlite and then connect to the database
import sqlite3

db = sqlite3.connect('mlflow.db')

In [15]:

#execute commands to check what are the tables
cur = db.cursor()
cur.execute("""select * from sqlite_master where type='table'""")
print(cur.fetchall())

[('table', 'experiments', 'experiments', 2, "CREATE TABLE experiments (\n\texperiment_id INTEGER NOT NULL, \n\tname VARCHAR(256) NOT NULL, \n\tartifact_location VARCHAR(256), \n\tlifecycle_stage VARCHAR(32), creation_time BIGINT, last_update_time BIGINT, \n\tCONSTRAINT experiment_pk PRIMARY KEY (experiment_id), \n\tCONSTRAINT experiments_lifecycle_stage CHECK (lifecycle_stage IN ('active', 'deleted')), \n\tUNIQUE (name)\n)"), ('table', 'alembic_version', 'alembic_version', 12, 'CREATE TABLE alembic_version (\n\tversion_num VARCHAR(32) NOT NULL, \n\tCONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)\n)'), ('table', 'experiment_tags', 'experiment_tags', 14, 'CREATE TABLE experiment_tags (\n\t"key" VARCHAR(250) NOT NULL, \n\tvalue VARCHAR(5000), \n\texperiment_id INTEGER NOT NULL, \n\tCONSTRAINT experiment_tag_pk PRIMARY KEY ("key", experiment_id), \n\tFOREIGN KEY(experiment_id) REFERENCES experiments (experiment_id)\n)'), ('table', 'tags', 'tags', 16, 'CREATE TABLE "tags" (\n\t"key

In [16]:
#Check all the tables
cur.execute("""SELECT * FROM experiments""")
cur.fetchall()

[(0, 'Default', 'mlflow-artifacts:/0', 'active', 1705588661834, 1705588661834),
 (1,
  'nyc-taxi-experiment',
  '/home/ubuntu/notebooks/mlruns/1',
  'active',
  1705591891060,
  1705591891060),
 (2,
  'nyc_taxi_experiment',
  '/home/ubuntu/notebooks/mlruns/2',
  'active',
  1705592485729,
  1705592485729),
 (3,
  '2_model_registry',
  'mlflow-artifacts:/3',
  'active',
  1706708938289,
  1706708938289),
 (4,
  'lightgbm prediction experiment',
  'mlflow-artifacts:/4',
  'active',
  1706710158792,
  1706710158792),
 (5,
  '2_homework',
  'mlflow-artifacts:/5',
  'active',
  1706856376705,
  1706856376705),
 (6,
  'random-forest-hyperopt',
  'mlflow-artifacts:/6',
  'deleted',
  1706861480031,
  1706863721127),
 (7,
  'random_forest_best_model',
  'mlflow-artifacts:/7',
  'deleted',
  1706862585292,
  1706862780840)]

In [17]:
#Then delete the concerned experiment

cur.execute("""DELETE FROM experiments WHERE experiment_id=6""")
cur.fetchall()

[]

In [18]:
cur.execute("""SELECT * FROM experiments""")
cur.fetchall()


#Finally check if our job is fulfilled

[(0, 'Default', 'mlflow-artifacts:/0', 'active', 1705588661834, 1705588661834),
 (1,
  'nyc-taxi-experiment',
  '/home/ubuntu/notebooks/mlruns/1',
  'active',
  1705591891060,
  1705591891060),
 (2,
  'nyc_taxi_experiment',
  '/home/ubuntu/notebooks/mlruns/2',
  'active',
  1705592485729,
  1705592485729),
 (3,
  '2_model_registry',
  'mlflow-artifacts:/3',
  'active',
  1706708938289,
  1706708938289),
 (4,
  'lightgbm prediction experiment',
  'mlflow-artifacts:/4',
  'active',
  1706710158792,
  1706710158792),
 (5,
  '2_homework',
  'mlflow-artifacts:/5',
  'active',
  1706856376705,
  1706856376705),
 (7,
  'random_forest_best_model',
  'mlflow-artifacts:/7',
  'deleted',
  1706862585292,
  1706862780840)]

Start of the mlflow workflow to check with the trials

In [ ]:

#create an experiment
mlflow.set_experiment("random-forest-hyperopt_1")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
for trial in trials:
    with mlflow.start_run():
        mlflow.set_tag("Trial No",trial.number)

        #logs
        mlflow.log_params(trial.params)

        model = RandomForestRegressor(**trial.params)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val,y_pred, squared=False)
        #log the artifact
        mlflow.sklearn.log_model(model,artifact_path="models")

        #log the metrics
        mlflow.log_metric('rmse',rmse)


In [ ]:

        #get the best model
best_model = RandomForestRegressor(**study.best_params)

        #log the best model
mlflow.sklearn.log_model(best_model,artifact_path="models")

        #eval the model
best_model.fit(X_train,y_train)
        y_pred = best_model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse',rmse)

    

In [25]:
from mlflow.client import MlflowClient
client = MlflowClient(tracking_uri="http://localhost:5000")


In [26]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/8', creation_time=1706888110103, experiment_id='8', last_update_time=1706888110103, lifecycle_stage='active', name='random-forest-hyperopt_1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1706856376705, experiment_id='5', last_update_time=1706856376705, lifecycle_stage='active', name='2_homework', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1706710158792, experiment_id='4', last_update_time=1706710158792, lifecycle_stage='active', name='lightgbm prediction experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1706708938289, experiment_id='3', last_update_time=1706708938289, lifecycle_stage='active', name='2_model_registry', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/2', creation_time=1705592485729, experiment_id='2', last_update_time=1705592485729, lifecycle_stage='active', name='nyc_taxi_experiment',

In [40]:
runs[0].data.tags['Trial No']


'9'

In [48]:
runs = client.search_runs(experiment_ids=8)
runs

[<Run: data=<RunData: metrics={'rmse': 2.4525004209611474}, params={'max_depth': '10',
  'min_samples_leaf': '2',
  'min_samples_split': '8',
  'n_estimators': '43'}, tags={'Trial No': '9',
  'mlflow.log-model.history': '[{"run_id": "3d50426d14a644f1855735d7e2286774", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2024-02-02 15:36:24.064327", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.11.5", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.3.2", '
                              '"serialization_format": "cloudpickle", "c

In [51]:
runs = client.search_runs(experiment_ids=8)
for run in runs:
    best_rmse = 1000
    best_trial = 0
    if run.data.metrics['rmse'] < best_rmse:
        best_rmse = run.data.metrics['rmse']
        best_trial = run.data.tags['Trial No']
    

print(f"The best rmse score is {best_rmse} for trial no {best_trial}")

The best rmse score is 2.4500656736687114 for trial no 0


---------------------------------------------------------

# Q5

In [1]:
##REGISTER THE BEST MODEL
import mlflow
from mlflow.client import MlflowClient
from mlflow.entities import ViewType




mlflow.set_tracking_uri(uri='http://localhost:5000')

In [2]:

client = MlflowClient(tracking_uri="http://localhost:5000")    

In [3]:
runs = client.search_runs(experiment_ids=8)
scores = dict()
runid = []
for run in runs:
    scores[run.info.run_id]=run.data.metrics['rmse']
    runid.append(run.info.run_id)

In [4]:
scores

{'3d50426d14a644f1855735d7e2286774': 2.4525004209611474,
 '4f065e83408f400b81e36546d3fe97a5': 2.483952634386172,
 '7495ae9c56eb490cac52d5843a983cb6': 2.4541242038507383,
 '4f02845e25e14b66bb4670fceb1becc4': 2.4535062634485025,
 'efd1d85059004a26a7ef297b4c2818b1': 2.4526381369438814,
 'ea3a6df6a92f4a5daf642701685b365d': 2.456601269454709,
 '1bdfd6bf5826476cbc370a66525e4b37': 2.4587050366466663,
 '2bcd2da272e5492d872e3547f2b1e089': 2.454700065963907,
 'ff38ba78e99445b089e6685a12ed246f': 2.454036643638431,
 '72fb9935fdef44d5aebf56a4edcd20cd': 2.4500656736687114}

In [5]:
sorted_scores=sorted(scores.items(),key=lambda item: item[1])
#Now we have sorted the items

In [8]:
#Here we will have to select the top 5 results

for i in sorted_scores[:5]:
    print(i)

('72fb9935fdef44d5aebf56a4edcd20cd', 2.4500656736687114)
('3d50426d14a644f1855735d7e2286774', 2.4525004209611474)
('efd1d85059004a26a7ef297b4c2818b1', 2.4526381369438814)
('4f02845e25e14b66bb4670fceb1becc4', 2.4535062634485025)
('ff38ba78e99445b089e6685a12ed246f', 2.454036643638431)


In [14]:
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

# Following illustrates commands to execute the model registration using mlflow client

In [21]:
for num, i in enumerate(sorted_scores[:5]):
    runs_uri = f"runs:/{i[0]}/models"
    model_source = RunsArtifactRepository.get_underlying_uri(runs_uri=runs_uri)
    model_name = f"model_new_{num}"
    client.create_registered_model(model_name)
    client.create_model_version(model_name,source=model_source, run_id=i[0])
    

2024/02/02 23:50:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_new_0, version 1
2024/02/02 23:50:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_new_1, version 1
2024/02/02 23:50:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_new_2, version 1
2024/02/02 23:50:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_new_3, version 1
2024/02/02 23:50:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_new_4, version 1


# Hence the Q5 has also been answered